In [25]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 5)
np.random.seed(1234)

!pip install git+https://github.com/brendanartley/Regressio --quiet

### Cubic Spline Article

In this article I will go through cubic splines, and show why they are a more robust solution than linear/polynommial regression. First I will walk through the mathematics behind cub splines, then I will explain Runge's phenomonon, and finally I will introduce [Regressio](https://github.com/brendanartley/Regressio). Regressio is a python library for univariate regression, interpolation and smoothing.

## Introduction



Firstly, cubic spline is a piecewise interpolation model that fits a cubic polynomial to each "piece" of the function. At the point where 2 polynomials meet, the 1st and 2nd derivatives are equal. This makes for a smooth fitting line.

For example, in the image above we can see how a cubic polynomial may be split. In this image there are 3 pieces, and the two central blue points are where polynomials intercect. We can see that the function is smooth around this point, and that the entire function is continous. Lets look at the mathematics behind fitting this regression line.

## Mathematics

Lets say we have three data points (2,3), (3,2), (4,4). When calculating the cubic spline betweens points, we need n-1 piecewise functions that regression models to the third degree.

Since we have 3 data points, we will need 2 piecewise functions. We will denote these as $f_1(x)$ and $f_2(x)$.

$$
f(x) = 
    \begin{cases}
        f_1(x) = a_1x_1^3 + b_1x_1^2 + c_1x_1 + d_1, \quad{}\text{for 2 <= f(x) <= 3} \\
        f_2(x) = a_1x_2^3 + b_1x_2^2 + c_1x_2 + d_1, \quad{}\text{for 3 <= f(x) <= 4}
    \end{cases}
$$

In each equation above we have 4 unknown variables (a, b, c, d). We will need to set up a system of equations to calculate for each equation, so therefore we have a total of 8 unknowns.

First, we know that for the 1st and 2nd points, these must fall on the first function.

$$ f_1(x) = a_1x_1^3 + b_1x_1^2 + c_1x_1 + d_1 = y_1$$

$$ f_1(x) = a_1x_2^3 + b_1x_2^2 + c_1x_2 + d_1 = y_2$$

Next, we know that the second and third points must fall on the second function.

$$ f_2(x) = a_2x_2^3 + b_2x_2^2 + c_2x_2 + d_2 = y_2$$

$$ f_2(x) = a_2x_3^3 + b_2x_3^2 + c_2x_3 + d_2 = y_3$$

We also know that the 1st derivatives of two functions must be equal when they intercept. This is when we are at the second data point.

$$ f_1'(x) = f_2'(x)$$

$$ 3a_1x_2^2 + 2b_1x_2 + c_1 = 3a_2x_2^2 + 2b_2x_1 + c_2$$

$$ 3a_1x_2^2 + 2b_1x_2 + c_1 - 3a_2x_2^2 - 2b_2x_1 - c_2 = 0$$

Then, we also know that the 2nd derviatives of the intersecting splines must be equal so we can form the equation.

$$ f_1''(x) = f_2''(x)$$

$$ 6a_1x_2 + 2b_1 = 6a_2x_2^2 + 2b_2$$

$$ 6a_1x_2 + 2b_1 - 6a_2x_2^2 - 2b_2 = 0$$

Finally, we want the 2nd derivatives at each endpoint to be 0. This makes the spline 'natural'.

$$ 6a_1x_1 + 2b_1 = 0$$

$$ 6a_2x_3 + 2b_2 = 0$$

The resulting 8 equations are as follows.

$$ a_1x_1^3 + b_1x_1^2 + c_1x_1 + d_1 = y_1$$

$$ a_1x_2^3 + b_1x_2^2 + c_1x_2 + d_1 = y_2$$

$$ a_2x_2^3 + b_2x_2^2 + c_2x_2 + d_2 = y_2$$

$$ a_2x_3^3 + b_2x_3^2 + c_2x_3 + d_2 = y_3$$

$$ 3a_1x_2^2 + 2b_1x_2 + c_1 - 3a_2x_2^2 - 2b_2x_1 - c_2 = 0$$

$$ 6a_1x_2 + 2b_1 - 6a_2x_2^2 - 2b_2 = 0$$

$$ 6a_1x_1 + 2b_1 = 0$$

$$ 6a_2x_3 + 2b_2 = 0$$

We can then plug in the three data points (2,3), (3,2), (4,4).

$$ 8a_1 + 4b_1 + 2c_1 + d_1 = 3$$

$$ 27a_1 + 9b_1 + 3c_1 + d_1 = 2$$

$$ 27a_2 + 9b_2 + 3c_2 + d_2 = 2$$

$$ 64a_2 + 16b_2 + 4c_2 + d_2 = 4$$

$$ 27a_1 + 6b_1 + c_1 - 27a_2 - 6b_2 - c_2 = 0$$

$$ 18a_1 + 2b_1 - 18a_2 - 2b_2 = 0$$

$$ 12a_1 + 2b_1 = 0$$

$$ 24a_2 + 2b_2 = 0$$

The equations above can then be represented in matrix form and solved using linear algebra. The equations are represented in a matrix of size 4 x (n - 1). In this example the matrix will be 8x8.

$$ A(x)\cdot{\overrightarrow{c(x)}} = \overrightarrow{b(x)} $$

$$
\begin{bmatrix} 
8 & 4 & 2 & 1 & 0 & 0 & 0 & 0 \\
27 & 9 & 3 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 27 & 9 & 3 & 1 \\
0 & 0 & 0 & 0 & 64 & 16 & 4 & 1 \\
27 & 6 & 1 & 0 & -27 & -6 & -1 & 0 \\
18 & 2 & 0 & 0 & -18 & -2 & 0 & 0 \\
12 & 2 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 24 & 2 & 0 & 0
\end{bmatrix} \cdot
\begin{bmatrix} 
a_1 \\ b_1 \\ c_1 \\ d_1 \\ a_2 \\ b_2 \\ b_3 \\ b_4
\end{bmatrix}
= 
\begin{bmatrix} 
3 \\ 2 \\ 2 \\ 4 \\ 0 \\ 0 \\ 0 \\ 0
\end{bmatrix}
$$

$$ \overrightarrow{c(x)} = A(x)^{-1}\cdot{b(x)} $$

$$ 
\overrightarrow{c(x)} =
\begin{bmatrix}
0.75 \\ -4.5 \\ 7.25 \\ 0.5 \\ -0.75 \\ 9 \\ -33.25 \\ 41
\end{bmatrix}
$$ 

We can now plug these values back into our two equations and we have the piecewise function!

$$
f(x) = 
    \begin{cases}
        f_1(x) = 0.75x^3 -4.5x^2 + 7.25x + 0.5, \quad{}\text{for 2 <= f(x) <= 3} \\
        f_2(x) = -0.75x^3 + 9x^2 - 33.25x + 41, \quad{}\text{for 3 <= f(x) <= 4}
    \end{cases}
$$

In [26]:
# Input Data
raw_x, raw_y = np.asarray([2,3,4]), np.asarray([3,2,4])

# Calculating Weights
b = np.array([3,2,2,4,0,0,0,0], dtype=np.float64)
A = np.array([[8,4,2,1,0,0,0,0],
              [27,9,3,1,0,0,0,0], 
              [0,0,0,0,27,9,3,1],
              [0,0,0,0,64,16,4,1],
              [27,6,1,0,-27,-6,-1,0],
              [18,2,0,0,-18,-2,0,0],
              [12,2,0,0,0,0,0,0],
              [0,0,0,0,24,2,0,0]], dtype=np.float64)

lines = np.dot(np.linalg.inv(A), b).reshape(-1, 4)

# Calculates x**0 + x**1 + x**2 + x**3
def plot(values, coeffs):
    # Coeffs are assumed to be in order 0, 1, ..., n-1
    expanded = np.hstack([coeffs[i] * (values ** i) for i in range(0, len(coeffs))])
    return np.sum(expanded, axis=1)

xs = np.linspace(raw_x.min(), raw_x.max(), 100)
y1s = plot(xs[xs<=3].reshape(-1,1), lines[0][::-1])
y2s = plot(xs[xs>3].reshape(-1,1), lines[1][::-1])
ys = np.concatenate([y1s, y2s])

plt.plot(xs, ys)
plt.scatter(raw_x, raw_y)
plt.show()

## Regressio Library

Now what happens when we have 200 points, and we want to fit 10 piecewise cubic polynomials?

This could do it by hand, but it is much easier to use an existing python library. We will use a package called [Regressio](https://github.com/brendanartley/Regressio) to do just that. In the following cell we are generating a random sample of 200 data points.

In [27]:
from regressio.datagen import generate_random_walk

x, y = generate_random_walk(200, plot=True)

Then, we can simply import a cubic spline model and fit the model to the data.

In [28]:
from regressio.models import cubic_spline

model = cubic_spline(pieces=10)
model.fit(x, y, plot=True)

This is pretty amazing model as we can fit highly variable relationships using mutliple low degree polynomials.

## Runge's Phenomenon

Fitting spline models was exactly what Carl David Tolmé Runge was doing in 1901, and he found that polynomial interpolation methods such as cubic spline outperformed linear regression models with high degrees. This is due to large osciallations at the edges of intervals in linear regression models. This is visualized nicely in this image by John D. Cook below.


This is significant because if we are given a data point that falls just outside of the boundaries of the training data, then small changes in x will result in very different predictions.

## Final Thoughts

Cubic spline is a more robust alternative to high degree linear regression models. I am suprised this is not taught directly after linear regression, as it addresses some of the model flaws. 

If you want to work through more examples of cubic spline, I encourage you to explore the [Regressio module](https://github.com/brendanartley/Regressio) and the source code. As the author of this package, I made the code base very readable for those trying to understand the models. 

And if you would like the notebook code for this article, you can find that [here](https://github.com/brendanartley/Medium-Article-Code).
